In [8]:
line = 'img_00091.jpg ,60.13121032714844 92.90408325195312 0.9837533831596375 ,52.727874755859375 103.09432983398438 1.0999332666397095 ,52.98760986328125 84.64228820800781 1.0287318229675293 ,60.1942138671875 116.5445556640625 0.9749239087104797 ,60.787384033203125 73.67532348632812 0.9833741188049316 ,112.78359985351562 139.41110229492188 1.001111626625061 ,112.98818969726562 55.78285217285156 1.0040918588638306 ,192.03021240234375 149.42005920410156 0.7704830169677734 ,191.98126220703125 42.03474426269531 0.8634825944900513 ,193.52606201171875 145.5717315673828 0.07413303107023239 ,193.52020263671875 39.55767822265625 0.1454618126153946 ,193.51556396484375 117.64739990234375 0.06861644983291626 ,193.522705078125 69.82029724121094 0.08380355685949326 ,193.5238037109375 -2.0425567626953125 0.038773953914642334 ,108.1351318359375 92.18673706054688 0.03272699937224388 ,193.51971435546875 151.5338897705078 0.04092157632112503 ,193.52301025390625 43.57696533203125 0.0669620931148529 , 32 66, 88 123, 89 137, 201 249, -16 137, 95 249, 30 36, 222 228'

In [9]:
line = [l for l in line.replace(',',"").split(' ') if l != '' and l != '\n']
imgpath = line[0]
line = line[1:]
line = [float(l) for l in line]

In [10]:
line

[60.13121032714844,
 92.90408325195312,
 0.9837533831596375,
 52.727874755859375,
 103.09432983398438,
 1.0999332666397095,
 52.98760986328125,
 84.64228820800781,
 1.0287318229675293,
 60.1942138671875,
 116.5445556640625,
 0.9749239087104797,
 60.787384033203125,
 73.67532348632812,
 0.9833741188049316,
 112.78359985351562,
 139.41110229492188,
 1.001111626625061,
 112.98818969726562,
 55.78285217285156,
 1.0040918588638306,
 192.03021240234375,
 149.42005920410156,
 0.7704830169677734,
 191.98126220703125,
 42.03474426269531,
 0.8634825944900513,
 193.52606201171875,
 145.5717315673828,
 0.07413303107023239,
 193.52020263671875,
 39.55767822265625,
 0.1454618126153946,
 193.51556396484375,
 117.64739990234375,
 0.06861644983291626,
 193.522705078125,
 69.82029724121094,
 0.08380355685949326,
 193.5238037109375,
 -2.0425567626953125,
 0.038773953914642334,
 108.1351318359375,
 92.18673706054688,
 0.03272699937224388,
 193.51971435546875,
 151.5338897705078,
 0.04092157632112503,
 193

In [11]:
        posepoints = line[0:51]
        head = line[51:55]
        lhand =line[55:59]
        rhand = line[59:63]
        bodybbox = line[63:]
        pose_values = dict()

In [15]:
bodybbox

[30.0, 36.0, 222.0, 228.0]

In [1]:
import numpy as np

In [7]:
class SampleFrames:
    """Sample frames from the video.
    Required keys are "total_frames", "start_index" , added or modified keys
    are "frame_inds", "frame_interval" and "num_clips".
    Args:
        clip_len (int): Frames of each sampled output clip.
        frame_interval (int): Temporal interval of adjacent sampled frames.
            Default: 1.
        num_clips (int): Number of clips to be sampled. Default: 1.
        temporal_jitter (bool): Whether to apply temporal jittering.
            Default: False.
        twice_sample (bool): Whether to use twice sample when testing.
            If set to True, it will sample frames with and without fixed shift,
            which is commonly used for testing in TSM model. Default: False.
        out_of_bound_opt (str): The way to deal with out of bounds frame
            indexes. Available options are 'loop', 'repeat_last'.
            Default: 'loop'.
        test_mode (bool): Store True when building test or validation dataset.
            Default: False.
        start_index (None): This argument is deprecated and moved to dataset
            class (``BaseDataset``, ``VideoDataset``, ``RawframeDataset``,
            etc), see this: https://github.com/open-mmlab/mmaction2/pull/89.
        keep_tail_frames (bool): Whether to keep tail frames when sampling.
            Default: False.
    """

    def __init__(self,
                 clip_len,
                 frame_interval=1,
                 num_clips=1,
                 temporal_jitter=False,
                 twice_sample=False,
                 out_of_bound_opt='loop',
                 test_mode=False,
                 start_index=None,
                 keep_tail_frames=False):

        self.clip_len = clip_len
        self.frame_interval = frame_interval
        self.num_clips = num_clips
        self.temporal_jitter = temporal_jitter
        self.twice_sample = twice_sample
        self.out_of_bound_opt = out_of_bound_opt
        self.test_mode = test_mode
        self.keep_tail_frames = keep_tail_frames
        assert self.out_of_bound_opt in ['loop', 'repeat_last']

        if start_index is not None:
            warnings.warn('No longer support "start_index" in "SampleFrames", '
                          'it should be set in dataset class, see this pr: '
                          'https://github.com/open-mmlab/mmaction2/pull/89')

    def _get_train_clips(self, num_frames):
        """Get clip offsets in train mode.
        It will calculate the average interval for selected frames,
        and randomly shift them within offsets between [0, avg_interval].
        If the total number of frames is smaller than clips num or origin
        frames length, it will return all zero indices.
        Args:
            num_frames (int): Total number of frame in the video.
        Returns:
            np.ndarray: Sampled frame indices in train mode.
        """
        ori_clip_len = self.clip_len * self.frame_interval

        if self.keep_tail_frames:
            avg_interval = (num_frames - ori_clip_len + 1) / float(
                self.num_clips)
            if num_frames > ori_clip_len - 1:
                base_offsets = np.arange(self.num_clips) * avg_interval
                clip_offsets = (base_offsets + np.random.uniform(
                    0, avg_interval, self.num_clips)).astype(int)
            else:
                clip_offsets = np.zeros((self.num_clips, ), dtype=int)
        else:
            avg_interval = (num_frames - ori_clip_len + 1) // self.num_clips

            if avg_interval > 0:
                base_offsets = np.arange(self.num_clips) * avg_interval
                clip_offsets = base_offsets + np.random.randint(
                    avg_interval, size=self.num_clips)
            elif num_frames > max(self.num_clips, ori_clip_len):
                clip_offsets = np.sort(
                    np.random.randint(
                        num_frames - ori_clip_len + 1, size=self.num_clips))
            elif avg_interval == 0:
                ratio = (num_frames - ori_clip_len + 1.0) / self.num_clips
                clip_offsets = np.around(np.arange(self.num_clips) * ratio)
            else:
                clip_offsets = np.zeros((self.num_clips, ), dtype=int)

        return clip_offsets

    def _get_test_clips(self, num_frames):
        """Get clip offsets in test mode.
        Calculate the average interval for selected frames, and shift them
        fixedly by avg_interval/2. If set twice_sample True, it will sample
        frames together without fixed shift. If the total number of frames is
        not enough, it will return all zero indices.
        Args:
            num_frames (int): Total number of frame in the video.
        Returns:
            np.ndarray: Sampled frame indices in test mode.
        """
        ori_clip_len = self.clip_len * self.frame_interval
        avg_interval = (num_frames - ori_clip_len + 1) / float(self.num_clips)
        if num_frames > ori_clip_len - 1:
            base_offsets = np.arange(self.num_clips) * avg_interval
            clip_offsets = (base_offsets + avg_interval / 2.0).astype(int)
            if self.twice_sample:
                clip_offsets = np.concatenate([clip_offsets, base_offsets])
        else:
            clip_offsets = np.zeros((self.num_clips, ), dtype=int)
        return clip_offsets

    def _sample_clips(self, num_frames):
        """Choose clip offsets for the video in a given mode.
        Args:
            num_frames (int): Total number of frame in the video.
        Returns:
            np.ndarray: Sampled frame indices.
        """
        if self.test_mode:
            clip_offsets = self._get_test_clips(num_frames)
        else:
            clip_offsets = self._get_train_clips(num_frames)

        return clip_offsets

    def __call__(self, results):
        """Perform the SampleFrames loading.
        Args:
            results (dict): The resulting dict to be modified and passed
                to the next transform in pipeline.
        """
        total_frames = results['total_frames']

        clip_offsets = self._sample_clips(total_frames)
        frame_inds = clip_offsets[:, None] + np.arange(
            self.clip_len)[None, :] * self.frame_interval
        frame_inds = np.concatenate(frame_inds)

        if self.temporal_jitter:
            perframe_offsets = np.random.randint(
                self.frame_interval, size=len(frame_inds))
            frame_inds += perframe_offsets

        frame_inds = frame_inds.reshape((-1, self.clip_len))
        if self.out_of_bound_opt == 'loop':
            frame_inds = np.mod(frame_inds, total_frames)
        elif self.out_of_bound_opt == 'repeat_last':
            safe_inds = frame_inds < total_frames
            unsafe_inds = 1 - safe_inds
            last_ind = np.max(safe_inds * frame_inds, axis=1)
            new_inds = (safe_inds * frame_inds + (unsafe_inds.T * last_ind).T)
            frame_inds = new_inds
        else:
            raise ValueError('Illegal out_of_bound option.')

        start_index = results['start_index']
        frame_inds = np.concatenate(frame_inds) + start_index
        results['frame_inds'] = frame_inds.astype(int)
        results['clip_len'] = self.clip_len
        results['frame_interval'] = self.frame_interval
        results['num_clips'] = self.num_clips
        return results

    def __repr__(self):
        repr_str = (f'{self.__class__.__name__}('
                    f'clip_len={self.clip_len}, '
                    f'frame_interval={self.frame_interval}, '
                    f'num_clips={self.num_clips}, '
                    f'temporal_jitter={self.temporal_jitter}, '
                    f'twice_sample={self.twice_sample}, '
                    f'out_of_bound_opt={self.out_of_bound_opt}, '
                    f'test_mode={self.test_mode})')
        return repr_str

In [8]:
results = dict(clip_len=1)
results['total_frames'] = 50
results['start_index'] = 1

In [9]:
sampleframes = SampleFrames(clip_len=32,
                           frame_interval=2,
                           num_clips=1)

In [10]:
x = sampleframes(results)

In [11]:
x['frame_inds']

array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
       35, 37, 39, 41, 43, 45, 47, 49,  1,  3,  5,  7,  9, 11, 13])